<a href="https://colab.research.google.com/github/Jualns/Neoplasia-Image-Classification/blob/main/An%C3%A1lise_do_Neoplasia_usando_YOLOv8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Contextualização

Revivi o projeto para tentar melhorar os modelos e também para tentar utilizar novas tecnologias.

Time original: Augusto Pereira, Dionizio, João Victor, Milena Andreuzo, Thalyta Genaro.
***

1. Projeto:
    - Consistiu em análise de dados e geração de modelos de IA para auxiliar no processo de classificação de imagens médicas para análise do estágio do câncer.

2. Objetivo final:
    - O objetivo principal é categorizar as imagens em **Muito** ou **Pouco** informativas em relação a quantidade de células cancerígenas, auxiliando na identificação de imagens adequadas para a análise subsequente do estágio do câncer.
3. Os 4 problemas das Imagens são:

1. Espaço em branco

<img align="left" src="https://github.com/Jualns/Neoplasia-Image-Classification/blob/main/Example-Problems/white.jpg?raw=true" width="1200">



2. Artefato aleatório
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img align="left" src="https://github.com/Jualns/Neoplasia-Image-Classification/blob/main/Example-Problems/random%20artefact.jpg?raw=true" width="1000">



3. Vasos com células de sangue
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
<img align="left" src="https://github.com/Jualns/Neoplasia-Image-Classification/blob/main/Example-Problems/blood%20vessel%20with%20blood%20cells.jpg?raw=true" width="1000">

4. Dobra de Tecido
<img align="left" src="https://github.com/Jualns/Neoplasia-Image-Classification/blob/main/Example-Problems/tissue%20fold.jpg?raw=true" width="1000">


# 2. Criação do dataset
***

Inicialmente precisamos criar o dataset com os problemas os vasos e com as dobras de tecido.

Para isso utilizei o [Roboflow](https://app.roboflow.com/cidamo-group-ufpr/neoplasiaimageclassification/overview). Detalhes em
[Links Úteis](#links-úteis).

A maior parte do projeto consistiu em ficar  dias classificando manualmente cada parte que gostaria de evitar.

**Segue o código para importar seus dados:**

In [1]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="4ba4QridEVlZrOVf6bTY")
project = rf.workspace("cidamo-group-ufpr").project("neoplasiaimageclassification")
dataset = project.version(1).download("yolov8")

!mkdir datasets
!mv NeoplasiaImageClassification-1/ datasets/.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.7/58.7 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 24.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 8.0 MB/s eta 0:00:00
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.1.1
    Uninstalling pyparsing-3.1.1:
      Successfully uninstalled pyparsing-3.1.1
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.8.0.76
    Uninstalling opencv-python-headless-4.8.0.76:
      Successfully uninstalled

loading Roboflow workspace...
loading Roboflow project...
[WARNING] we noticed you are downloading a `yolov8` datasets but you don't have `ultralytics` installed. Roboflow `.deploy` supports only models trained with `ultralytics==8.0.134`, to intall it `pip install ultralytics==8.0.134`.



Extracting Dataset Version Zip to NeoplasiaImageClassification-1 in yolov8:: 100%|██████████| 712/712 [00:00<00:00, 7834.84it/s]


**ALTERAR O "data.yaml" POIS VEM COM PROBLEMA**

# 3. Configurações do ambiente da YOLO
***
Pip install `ultralytics` e [dependências](https://github.com/ultralytics/ultralytics/blob/main/requirements.txt).

In [2]:
%pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.182 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 26.5/78.2 GB disk)


## 3.1 Processo básico

<p align=""><a href="https://bit.ly/ultralytics_hub"><img width="1000" src="https://github.com/ultralytics/assets/raw/main/yolov8/banner-integrations.png"/></a></p>

Veja a documentação [YOLOv8 Train Docs](https://docs.ultralytics.com/modes/train/) para mais informações.

# Plataformas "Loggers"

São plataformas que vão registrar muitas informações úteis como:
- Log
- Melhor e Última versão do modelo treinado
- Estatísticas gerais e específicas
- Exemplo de imagens de treino e teste

Basicamente salvam tudo que você precisa para seguir com seu modelo.


- [Comet](https://www.comet.com/jualns/general/view/new/panels)
- [ClearMl](app.clear.ml/)

Observação: Sim é possível usar as duas plataformas ao mesmo tempo

In [3]:
#@title Exemplo usando o Comet para o YOLO logger {run: 'auto'}
logger = 'Comet' #@param ['Comet', 'TensorBoard']

if logger == 'Comet':
  %pip install -q comet_ml
  import comet_ml; comet_ml.init()
elif logger == 'TensorBoard':
  %load_ext tensorboard
  %tensorboard --logdir .

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.9/561.9 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 61.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.8/224.8 kB 24.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.2/512.2 kB 43.6 MB/s eta 0:00:00
Please paste your Comet API key from https://www.comet.com/api/my/settings/
(api key may not show as you type)
Comet API key: ··········


COMET INFO: Valid Comet API Key saved in /root/.comet.config (set COMET_CONFIG to change where it is saved).


In [ ]:
#@title Exemplo usando o ClearML para o YOLO logger {run: 'auto'}
logger = 'ClearML' #@param ['Comet', 'ClearML', 'TensorBoard']

if logger == 'Comet':
  %pip install -q comet_ml
  import comet_ml; comet_ml.init()
elif logger == 'ClearML':
  %pip install -q clearml
  import clearml; clearml.browser_login()
elif logger == 'TensorBoard':
  %load_ext tensorboard
  %tensorboard --logdir runs/train

# 4. Processo de treinamento da YOLO com dados personalizados

In [ ]:
# Treinar o YOLOv8n nos dados personalizados direto no terminal
!yolo train model=yolov8m.pt data=/content/datasets/NeoplasiaImageClassification-1/data.yaml epochs=3 imgsz=640

Ultralytics YOLOv8.0.178 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/datasets/NeoplasiaImageClassification-1/data.yaml, epochs=3, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=

In [ ]:
# Treinar o YOLOv8n nos dados personalizados direto no terminal
!yolo train model=yolov8m.pt data=/content/datasets/NeoplasiaImageClassification-1/data.yaml epochs=300 imgsz=640

100% 49.7M/49.7M [00:00<00:00, 78.9MB/s]
Ultralytics YOLOv8.0.176 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/datasets/NeoplasiaImageClassification-1/data.yaml, epochs=300, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=No

# 4. Exportando seu Modelo

Você pode exportar seu modelo baseado no YOLOv8 para os formatos abaixo apenas usando o argumento `format`, i.e. `format=onnx`. Entre no [YOLOv8 Export Docs](https://docs.ultralytics.com/modes/export/) para detalhes.

- 💡 ProTip: Export to [ONNX](https://onnx.ai/) or [OpenVINO](https://docs.openvino.ai/latest/index.html) for up to 3x CPU speedup.  
- 💡 ProTip: Export to [TensorRT](https://developer.nvidia.com/tensorrt) for up to 5x GPU speedup.


| Format                                                             | `format` Argument | Model                     | Metadata | Arguments                                           |
|--------------------------------------------------------------------|-------------------|---------------------------|----------|-----------------------------------------------------|
| [PyTorch](https://pytorch.org/)                                    | -                 | `yolov8n.pt`              | ✅        | -                                                   |
| [TorchScript](https://pytorch.org/docs/stable/jit.html)            | `torchscript`     | `yolov8n.torchscript`     | ✅        | `imgsz`, `optimize`                                 |
| [ONNX](https://onnx.ai/)                                           | `onnx`            | `yolov8n.onnx`            | ✅        | `imgsz`, `half`, `dynamic`, `simplify`, `opset`     |
| [OpenVINO](https://docs.openvino.ai/latest/index.html)             | `openvino`        | `yolov8n_openvino_model/` | ✅        | `imgsz`, `half`                                     |
| [TensorRT](https://developer.nvidia.com/tensorrt)                  | `engine`          | `yolov8n.engine`          | ✅        | `imgsz`, `half`, `dynamic`, `simplify`, `workspace` |
| [CoreML](https://github.com/apple/coremltools)                     | `coreml`          | `yolov8n.mlpackage`       | ✅        | `imgsz`, `half`, `int8`, `nms`                      |
| [TF SavedModel](https://www.tensorflow.org/guide/saved_model)      | `saved_model`     | `yolov8n_saved_model/`    | ✅        | `imgsz`, `keras`                                    |
| [TF GraphDef](https://www.tensorflow.org/api_docs/python/tf/Graph) | `pb`              | `yolov8n.pb`              | ❌        | `imgsz`                                             |
| [TF Lite](https://www.tensorflow.org/lite)                         | `tflite`          | `yolov8n.tflite`          | ✅        | `imgsz`, `half`, `int8`                             |
| [TF Edge TPU](https://coral.ai/docs/edgetpu/models-intro/)         | `edgetpu`         | `yolov8n_edgetpu.tflite`  | ✅        | `imgsz`                                             |
| [TF.js](https://www.tensorflow.org/js)                             | `tfjs`            | `yolov8n_web_model/`      | ✅        | `imgsz`                                             |
| [PaddlePaddle](https://github.com/PaddlePaddle)                    | `paddle`          | `yolov8n_paddle_model/`   | ✅        | `imgsz`                                             |
| [ncnn](https://github.com/Tencent/ncnn)                            | `ncnn`            | `yolov8n_ncnn_model/`     | ✅        | `imgsz`, `half`                                     |


In [ ]:
!yolo export model=yolov8m.pt format=torchscript

Ultralytics YOLOv8.0.176 🚀 Python-3.10.12 torch-2.0.1+cu118 CPU (Intel Xeon 2.20GHz)
YOLOv8m summary (fused): 218 layers, 25886080 parameters, 0 gradients

PyTorch: starting from 'yolov8m.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 84, 8400) (49.7 MB)

TorchScript: starting export with torch 2.0.1+cu118...
TorchScript: export success ✅ 8.9s, saved as 'yolov8m.torchscript' (99.2 MB)

Export complete (13.1s)
Results saved to /content
Predict:         yolo predict task=detect model=yolov8m.torchscript imgsz=640  
Validate:        yolo val task=detect model=yolov8m.torchscript imgsz=640 data=coco.yaml  
Visualize:       https://netron.app
💡 Learn more at https://docs.ultralytics.com/modes/export


# 5. Código em Python

O YOLOv8 foi reimaginado usando os princípios do Python para a experiência YOLO do Python mais perfeita até agora. Os modelos YOLOv8 podem ser carregados a partir de um ponto de verificação treinado ou criados do zero. Em seguida, métodos são usados para treinar, avaliar, prever e exportar o modelo. Veja exemplos detalhados de uso do Python na [documentação do YOLOv8 sobre Python](https://docs.ultralytics.com/usage/python/).

In [ ]:
from ultralytics import YOLO

# Lendo o modelo
#model = YOLO('yolov8n.yaml')  # Criando um modelo "vazio"
model = YOLO('yolov8m.pt')  # Lendo um modelo pré-treinado (recommended for training)

# Usando o modelo
results = model.train(data='/content/datasets/NeoplasiaImageClassification-1/data.yaml', epochs=3)  # Treinando o modelo
results = model.val()  # Avaliando a performance nos dados de Validação
results = model('/content/datasets/NeoplasiaImageClassification-1/test/images/1A_s0c0x10664-1600y2154-1200m11_jpg.rf.bbca9452feeb8b93fecf0adeb9843461.jpg')  # Prevendo uma imagem
results = model.export(format='onnx')  # Exportando para o formato ONNX

Ultralytics YOLOv8.0.178 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/datasets/NeoplasiaImageClassification-1/data.yaml, epochs=3, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, boxes=True, format=

ClearML Task: created new task id=0df35023edbe422bb9766a68c48b74ea


ClearML results page: https://app.clear.ml/projects/16c2bf414bbb48b9bdfed3fcb851eb55/experiments/0df35023edbe422bb9766a68c48b74ea/output/log


ClearML Initialized a new task. If you want to run remotely, please add clearml-init and connect your arguments before initializing YOLO.
COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: fastai, sklearn, torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Couldn't find a Git repository in '/content' nor in any parent directory. Set `COMET_GIT_DIRECTORY` if your Git Repository is elsewhere.
COMET INFO: Experiment is live on comet.com https://www.comet.com/jualns/general/f97f4c020ab842ef9be1e37a880b843e

TensorBoard: Start with 'tensorboard --logdir runs/detect/train4', view at http://localhost:6006/


2023-09-14 23:53:24,521 - clearml.Task - INFO - Storing jupyter notebook directly as code


Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks with YOLOv8n...
AMP: checks passed ✅
train: Scanning /content/datasets/NeoplasiaImageClassification-1/train/labels.cache... 245 images, 1 backgrounds, 0 corrupt: 100%|██████████| 245/245 [00:00<?, ?it/s]
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))
val: Scanning /content/datasets/NeoplasiaImageClassification-1/valid/labels.cache... 57 images, 1 backgrounds, 0 corrupt: 100%|██████████| 57/57 [00:00<?, ?it/s]
Plotting labels to runs/detect/train4/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 77 weight(decay=0.0), 84 weight(decay=0.0005), 83 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 2 datal

2023-09-14 23:55:08,871 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/YOLOv8/colab.0df35023edbe422bb9766a68c48b74ea/models/best.pt


COMET INFO: Please wait for metadata to finish uploading (timeout is 3600 seconds)
Ultralytics YOLOv8.0.178 🚀 Python-3.10.12 torch-2.0.1+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25840918 parameters, 0 gradients
val: Scanning /content/datasets/NeoplasiaImageClassification-1/valid/labels.cache... 57 images, 1 backgrounds, 0 corrupt: 100%|██████████| 57/57 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:09<00:00,  2.50s/it]
                   all         57        632      0.208      0.174      0.175     0.0736
             BloodCell         57        153          0          0    0.00129   0.000773
            TissueFold         57        479      0.417      0.349      0.348      0.146
Speed: 7.7ms preprocess, 36.5ms inference, 0.0ms loss, 14.1ms postprocess per image
Results saved to runs/detect/val

image 1/1 /content/datasets/NeoplasiaImageClassification-1/test/images/1A_s0c0

============= Diagnostic Run torch.onnx.export version 2.0.1+cu118 =============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



ONNX: export success ✅ 14.5s, saved as 'runs/detect/train4/weights/best.onnx' (98.8 MB)

Export complete (17.8s)
Results saved to /content/runs/detect/train4/weights
Predict:         yolo predict task=detect model=runs/detect/train4/weights/best.onnx imgsz=640  
Validate:        yolo val task=detect model=runs/detect/train4/weights/best.onnx imgsz=640 data=/content/datasets/NeoplasiaImageClassification-1/data.yaml  
Visualize:       https://netron.app


In [ ]:
pred = model('/content/datasets/NeoplasiaImageClassification-1/train/images/1A_s0c0x10665-1600y14024-982m133_jpg.rf.8858558a4a5627bf477c0a35e2ca725b.jpg')
pred


image 1/1 /content/datasets/NeoplasiaImageClassification-1/train/images/1A_s0c0x10665-1600y14024-982m133_jpg.rf.8858558a4a5627bf477c0a35e2ca725b.jpg: 640x640 1 BloodCell, 48 TissueFolds, 40.3ms
Speed: 7.4ms preprocess, 40.3ms inference, 3.3ms postprocess per image at shape (1, 3, 640, 640)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'BloodCell', 1: 'TissueFold'}
 orig_img: array([[[190, 192, 203],
         [179, 178, 188],
         [166, 158, 168],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[193, 195, 206],
         [180, 179, 189],
         [168, 160, 170],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[195, 199, 210],
         [181, 182, 192],
         [171, 163, 173],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        ...,
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         [255, 255, 255],
         [255, 255, 255],
         [255, 255, 255]],
 
        [[255, 255, 255],
         [255, 255, 255],
         [255, 255, 255],
         ...,
         

In [ ]:
pred[0].boxes.data

tensor([[4.9413e+02, 1.3390e+02, 5.3885e+02, 1.9188e+02, 9.6671e-01, 1.0000e+00],
        [2.5601e+02, 1.1905e+02, 3.2089e+02, 2.5448e+02, 9.5547e-01, 1.0000e+00],
        [3.8677e+02, 2.6677e+01, 4.8205e+02, 1.6410e+02, 9.5377e-01, 1.0000e+00],
        [4.1687e+02, 1.9802e+01, 4.8596e+02, 1.2766e+02, 9.3531e-01, 1.0000e+00],
        [8.7544e+01, 1.6613e+02, 1.4376e+02, 3.0333e+02, 9.3062e-01, 1.0000e+00],
        [3.3985e+01, 1.5929e+02, 1.3937e+02, 2.6327e+02, 9.2381e-01, 1.0000e+00],
        [3.6373e+01, 1.5225e+02, 1.4529e+02, 2.9725e+02, 8.5773e-01, 1.0000e+00],
        [4.9105e+02, 0.0000e+00, 5.8699e+02, 6.7830e+01, 8.3906e-01, 1.0000e+00],
        [3.3399e+01, 1.6089e+02, 1.0699e+02, 2.6651e+02, 8.3747e-01, 1.0000e+00],
        [4.0752e+02, 3.3439e+02, 6.4000e+02, 6.4000e+02, 8.3439e-01, 0.0000e+00],
        [2.5560e+02, 9.5193e+01, 3.6430e+02, 2.4313e+02, 8.2036e-01, 1.0000e+00],
        [2.5438e+02, 8.3882e+01, 3.6529e+02, 2.0554e+02, 8.1729e-01, 1.0000e+00],
        [0.0000e

# 6. O que o YOLO faz?

Com o YOLOv8 você pode treinar, validar, predizer e exportar modelos para as tarefas mais comuns de visão cumputacional com IA: [Detect](https://docs.ultralytics.com/tasks/detect/), [Segment](https://docs.ultralytics.com/tasks/segment/), [Classify](https://docs.ultralytics.com/tasks/classify/) e [Pose](https://docs.ultralytics.com/tasks/pose/).

<br><img width="1024" src="https://raw.githubusercontent.com/ultralytics/assets/main/im/banner-tasks.png">


# Links Úteis

## Criando seu Dataset Personalizado

Tutorial:
- [Data Annotation Object Detection (YOLO)?](https://muhammadrizwanmunawar.medium.com/labeling-data-for-object-detection-yolo-5a4fa4f05844)

Aplicativo (Local e limitado):
- [Label Studio](https://labelstud.io/blog/quickly-create-datasets-for-training-yolo-object-detection-with-label-studio/)

Online (Hospedado e Grátis):
- [Roboflow](https://app.roboflow.com)

## Treinando o YOLO com dados personalizados
- [Train YOLOv8 on Custom Data?](https://muhammadrizwanmunawar.medium.com/train-yolov8-on-custom-data-6d28cd348262)
- [Train YOLOv8 on Custom Dataset A Complete Tutorial](https://learnopencv.com/train-yolov8-on-custom-dataset/)